
# Week 2: Advanced EDA + Baseline ML Models for AQI Forecasting

This notebook includes:
- Week 1 steps (data preparation, AQI calculation, safe/unsafe hours)
- Week 2 tasks:
  - Advanced EDA
  - Baseline ML models (Persistence, Moving Average)
  - Evaluation metrics (RMSE, MAE, R²)
  - Export of engineered features for Week 3


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:

# Load dataset (Mumbai from city_hour.csv)
file_path = "/content/city_hour.csv"   # Update if needed
df = pd.read_csv(file_path, low_memory=False)

# Convert datetime
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

# Filter Mumbai
df_mumbai = df[df['City'].str.lower() == "mumbai"].copy()
df_mumbai = df_mumbai.set_index('Datetime').sort_index()

# Select pollutants
pollutants = ['PM2.5','PM10','NO2','O3','CO','SO2']
df_mumbai = df_mumbai[pollutants]

# Interpolate missing values
df_clean = df_mumbai.interpolate(method="time", limit=6)
df_clean.head()


In [ ]:

# Breakpoints for PM2.5 and PM10 (simplified CPCB)
bp = {
    'PM2.5': [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,500,401,500)],
    'PM10':  [(0,50,0,50),(51,100,51,100),(101,250,101,200),(251,350,201,300),(351,430,301,400),(431,1000,401,500)]
}

def sub_index(conc, pollutant):
    if pd.isna(conc) or pollutant not in bp: return None
    for (bp_lo,bp_hi,i_lo,i_hi) in bp[pollutant]:
        if bp_lo <= conc <= bp_hi:
            return ((i_hi-i_lo)/(bp_hi-bp_lo))*(conc-bp_lo)+i_lo
    return None

# Compute AQI
df_clean['pm25_si'] = df_clean['PM2.5'].apply(lambda x: sub_index(x,'PM2.5'))
df_clean['pm10_si'] = df_clean['PM10'].apply(lambda x: sub_index(x,'PM10'))
df_clean['AQI'] = df_clean[['pm25_si','pm10_si']].max(axis=1, skipna=True)

# AQI categories
def aqi_category(aqi):
    if pd.isna(aqi): return None
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Satisfactory"
    elif aqi <= 200: return "Moderate"
    elif aqi <= 300: return "Poor"
    elif aqi <= 400: return "Very Poor"
    else: return "Severe"

df_clean['AQI_category'] = df_clean['AQI'].apply(aqi_category)
df_clean['is_safe_hour'] = df_clean['AQI'] <= 100
df_clean.head()


In [ ]:

# Time-based features
df_clean['hour'] = df_clean.index.hour
df_clean['weekday'] = df_clean.index.weekday
df_clean['month'] = df_clean.index.month

# Lag features
for lag in [1,2,3,6,12,24]:
    df_clean[f'AQI_lag{lag}'] = df_clean['AQI'].shift(lag)

# Rolling averages
for window in [3,6,12,24]:
    df_clean[f'AQI_roll{window}'] = df_clean['AQI'].rolling(window).mean()

# Drop NA after feature creation
df_model = df_clean.dropna()

# Export features for Week 3
df_model.to_csv("aqi_features.csv")
print("Exported aqi_features.csv with", df_model.shape[0], "rows")


In [ ]:

sns.set(style="whitegrid")

# AQI over time
plt.figure(figsize=(14,4))
df_clean['AQI'].plot(title="AQI over Time (Mumbai)")
plt.ylabel("AQI")
plt.show()

# Distribution
plt.figure(figsize=(8,4))
sns.histplot(df_clean['AQI'].dropna(), bins=40, kde=True)
plt.title("Distribution of AQI (Mumbai)")
plt.show()

# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(df_clean[pollutants+['AQI']].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap: Pollutants vs AQI")
plt.show()

# Weekly pattern
plt.figure(figsize=(8,4))
df_clean.groupby('weekday')['AQI'].mean().plot(marker='o')
plt.title("Average AQI by Weekday (0=Mon)")
plt.ylabel("AQI")
plt.show()


In [ ]:

from sklearn.model_selection import train_test_split

# Train-test split (last 30% as test)
split_idx = int(len(df_model)*0.7)
train, test = df_model.iloc[:split_idx], df_model.iloc[split_idx:]

# Define true values
y_true = test['AQI']

# Model 1: Persistence (t+1 = t)
y_pred_persist = test['AQI_lag1']
rmse_persist = np.sqrt(mean_squared_error(y_true, y_pred_persist))
mae_persist = mean_absolute_error(y_true, y_pred_persist)
r2_persist = r2_score(y_true, y_pred_persist)

# Model 2: Moving Average (24h)
y_pred_ma24 = test['AQI_roll24']
rmse_ma24 = np.sqrt(mean_squared_error(y_true, y_pred_ma24))
mae_ma24 = mean_absolute_error(y_true, y_pred_ma24)
r2_ma24 = r2_score(y_true, y_pred_ma24)

print("Baseline Model Performance:")
print(f"Persistence → RMSE: {rmse_persist:.2f}, MAE: {mae_persist:.2f}, R²: {r2_persist:.2f}")
print(f"Moving Avg (24h) → RMSE: {rmse_ma24:.2f}, MAE: {mae_ma24:.2f}, R²: {r2_ma24:.2f}")
